In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.5 MB/s eta 0:00:00


In [3]:
fashion_mnist = keras.datasets.fashion_mnist

In [4]:
(train_images, train_labels), (test_images, test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 1s 0us/step


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images), 28, 28,  1)
test_images=test_images.reshape(len(test_images), 28, 28, 1)

In [8]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

<ipython-input-9-2bbfbe7dae29>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import RandomSearch


In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 29s]
val_accuracy: 0.9068333506584167

Best val_accuracy So Far: 0.9139999747276306
Total elapsed time: 00h 02m 49s


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 64)        51264     
                                                                 
 flatten (Flatten)           (None, 30976)             0         
                                                                 
 dense (Dense)               (None, 64)                1982528   
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 2,034,762
Trainable params: 2,034,762
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 9s 4ms/step - loss: 0.1411 - accuracy: 0.9472 - val_loss: 0.2478 - val_accuracy: 0.9142
Epoch 5/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1057 - accuracy: 0.9607 - val_loss: 0.2843 - val_accuracy: 0.9128
Epoch 6/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0747 - accuracy: 0.9729 - val_loss: 0.2872 - val_accuracy: 0.9172
Epoch 7/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0529 - accuracy: 0.9803 - val_loss: 0.3346 - val_accuracy: 0.9168
Epoch 8/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0400 - accuracy: 0.9854 - val_loss: 0.4474 - val_accuracy: 0.9127
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0347 - accuracy: 0.9878 - val_loss: 0.4459 - val_accuracy: 0.9118
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0263 - accuracy: 0.9905 - val_loss: 0.4842 - val_accuracy